# DRAM I-V Prediction using Neural Network

DRAM 소자의 I-V 특성을 예측하는 신경망 모델

**데이터**: 1718개 샘플, 5개 특성  
**입력**: Spacer(nm), Doping(cm⁻³), Gate(nm), Voltage(V)  
**출력**: Current(A)


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

import warnings
warnings.filterwarnings('ignore')

# 시드 설정
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

# 디바이스
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')


## 1. 데이터 로딩


In [ ]:
# 데이터 로딩
data = np.load('dram_data.npy')

print(f"Shape: {data.shape}")
print(f"\n칼럼:")
print(f"  [0] spacer_length (nm)")
print(f"  [1] doping_conc (cm⁻³)")
print(f"  [2] gate_length (nm)")
print(f"  [3] voltage (V)")
print(f"  [4] current (A)")

# 처음 5개 확인
print(f"\n처음 5개 샘플:")
print(data[:5])


## 2. 데이터 전처리


In [ ]:
# Signed Log 변환 함수
def signed_log(x, eps=1e-60):
    return np.sign(x) * np.log10(np.abs(x) + eps)

def inverse_signed_log(y, eps=1e-60):
    return np.sign(y) * (10 ** np.abs(y) - eps)

# 입력/출력 분리
X = data[:, :4].copy()  # spacer, doping, gate, voltage
y = data[:, 4:5].copy()  # current

# 변환
X[:, 1] = np.log10(X[:, 1])  # doping → log10
y = signed_log(y)             # current → signed log

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")
print(f"\n변환 후 범위:")
print(f"  Spacer: [{X[:, 0].min():.1f}, {X[:, 0].max():.1f}]")
print(f"  Doping(log): [{X[:, 1].min():.1f}, {X[:, 1].max():.1f}]")
print(f"  Gate: [{X[:, 2].min():.1f}, {X[:, 2].max():.1f}]")
print(f"  Voltage: [{X[:, 3].min():.1f}, {X[:, 3].max():.1f}]")
print(f"  Current(log): [{y.min():.1f}, {y.max():.1f}]")


In [ ]:
# 정규화
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)

# Train/Val/Test 분할
X_temp, X_test, y_temp, y_test = train_test_split(X_scaled, y_scaled, test_size=0.15, random_state=SEED)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.15/0.85, random_state=SEED)

print(f"Train: {len(X_train)} ({len(X_train)/len(X)*100:.1f}%)")
print(f"Val:   {len(X_val)} ({len(X_val)/len(X)*100:.1f}%)")
print(f"Test:  {len(X_test)} ({len(X_test)/len(X)*100:.1f}%)")


In [ ]:
# PyTorch 텐서 변환
X_train_t = torch.FloatTensor(X_train).to(device)
y_train_t = torch.FloatTensor(y_train).to(device)
X_val_t = torch.FloatTensor(X_val).to(device)
y_val_t = torch.FloatTensor(y_val).to(device)
X_test_t = torch.FloatTensor(X_test).to(device)
y_test_t = torch.FloatTensor(y_test).to(device)

# DataLoader
BATCH_SIZE = 32
train_loader = DataLoader(TensorDataset(X_train_t, y_train_t), batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(TensorDataset(X_val_t, y_val_t), batch_size=BATCH_SIZE, shuffle=False)

print(f"Batch size: {BATCH_SIZE}")
print(f"Train batches: {len(train_loader)}")


ㄹ 

In [ ]:
class DRAM_NN(nn.Module):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(4, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 1)
        )
    
    def forward(self, x):
        return self.network(x)

model = DRAM_NN().to(device)
print(model)
print(f"\n파라미터: {sum(p.numel() for p in model.parameters()):,}")


## 4. 학습


In [ ]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10)

NUM_EPOCHS = 200
best_val_loss = float('inf')
patience = 0
EARLY_STOP = 30

history = {'train_loss': [], 'val_loss': [], 'lr': []}

print("학습 시작...")
for epoch in range(NUM_EPOCHS):
    # Train
    model.train()
    train_loss = 0
    for X_batch, y_batch in train_loader:
        pred = model(X_batch)
        loss = criterion(pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * len(X_batch)
    train_loss /= len(train_loader.dataset)
    
    # Validation
    model.eval()
    with torch.no_grad():
        val_pred = model(X_val_t)
        val_loss = criterion(val_pred, y_val_t).item()
    
    history['train_loss'].append(train_loss)
    history['val_loss'].append(val_loss)
    history['lr'].append(optimizer.param_groups[0]['lr'])
    
    scheduler.step(val_loss)
    
    if (epoch + 1) % 20 == 0:
        print(f"Epoch {epoch+1:3d} | Train: {train_loss:.6f} | Val: {val_loss:.6f} | LR: {optimizer.param_groups[0]['lr']:.6f}")
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_model.pth')
        patience = 0
    else:
        patience += 1
    
    if patience >= EARLY_STOP:
        print(f"Early stopping at epoch {epoch+1}")
        break

print(f"\n✅ 학습 완료! Best val loss: {best_val_loss:.6f}")


In [ ]:
# 학습 곡선
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

axes[0].plot(history['train_loss'], label='Train')
axes[0].plot(history['val_loss'], label='Val')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Loss Curve')
axes[0].set_yscale('log')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(history['lr'], color='red')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Learning Rate')
axes[1].set_title('LR Schedule')
axes[1].set_yscale('log')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()
